In [3]:
import pandas as pd
import re

In [4]:
df = pd.read_csv("https://minio.lab.sspcloud.fr/mligeret1/constructions_non_resid_geocoded.csv",sep=",",on_bad_lines="warn")

/tmp/ipykernel_25864/1214635194.py:1: DtypeWarning: Columns (1,3,5,8,12,18,20,21,22,24,30,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,108,114,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("https://minio.lab.sspcloud.fr/mligeret1/constructions_non_resid_geocoded.csv",sep=",",on_bad_lines="warn")


In [5]:
df.head()

,Unnamed: 0,Code de la région du lieu des travaux - Code de la zone,Code de la région du lieu des travaux - Libellé de la zone,Code du département du lieu des travaux - Code de la zone,Code du département du lieu des travaux - Libellé de la zone,Code de la commune du lieu des travaux,"Type de DAU -- demande d'autorisation d'urbanisme (PC ou DP généralement, rarement PA)",Numéro d'enregistrement de la DAU,Etat d'avancement du projet,Date réelle d'autorisation (PC) ou de non-opposition (DP) initiale,...,result_name,result_street,result_postcode,result_city,result_context,result_citycode,result_oldcitycode,result_oldcity,result_district,result_status
0,0,REG_CODE,REG_LIBELLE,DEP_CODE,DEP_LIBELLE,COMM,TYPE_DAU,NUM_DAU,ETAT_DAU,DATE_REELLE_AUTORISATION,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not-found
1,1,82,Rhône-Alpes,01,Ain,01001,PC,00100117V0002,5,2017-05-05,...,L'Abergement-Clémenciat,NaN,1400.0,L'Abergement-Clémenciat,"01, Ain, Auvergne-Rhône-Alpes",01001,NaN,NaN,NaN,ok
2,2,82,Rhône-Alpes,01,Ain,01001,PC,00100119V0005,2,2019-08-29,...,203 Rue des 5 Chemins,Rue des 5 Chemins,1400.0,L'Abergement-Clémenciat,"01, Ain, Auvergne-Rhône-Alpes",01001,NaN,NaN,NaN,ok
3,3,82,Rhône-Alpes,01,Ain,01001,PC,00100120V0003,5,2020-05-25,...,172 Route du Château,Route du Château,1400.0,L'Abergement-Clémenciat,"01, Ain, Auvergne-Rhône-Alpes",01001,NaN,NaN,NaN,ok
4,4,82,Rhône-Alpes,01,Ain,01001,DP,00100119V0011,2,2019-07-22,...,Chemin sous Brody,Chemin sous Brody,1400.0,L'Abergement-Clémenciat,"01, Ain, Auvergne-Rhône-Alpes",01001,NaN,NaN,NaN,ok


In [6]:
dico_variables = pd.read_excel("https://minio.lab.sspcloud.fr/mligeret1/dictionnaire-variables-locaux-permis-construire-20220531.xls")

In [7]:
dico_variables.head()

,Nom de la variable,Description de la variable,Nouveau mai 2022?,Modalités,Format,Observations,Origine
0,REG,Code de la région du lieu des travaux,NaN,Nomenclature du Code Officiel Géographique (CO...,"Alphanumérique, 2 caractères",NaN,Numéro d'enregistrement de la DAU
1,DEP,Code du département du lieu des travaux,NaN,NaN,"Alphanumérique, 2 caractères",NaN,Numéro d'enregistrement de la DAU
2,COMM,Code de la commune du lieu des travaux,NaN,Nomenclature du Code Officiel Géographique (CO...,"Alphanumérique, 5 caractères",Le code vient directement du numéro d'enregist...,Numéro d'enregistrement de la DAU
3,Type_DAU,Type de DAU -- demande d'autorisation d'urbani...,NaN,PC (Permis de construire)\nDP (Déclaration Pré...,"Alphanumérique, 2 caractères",NaN,Numéro d'enregistrement de la DAU
4,Num_DAU,Numéro d'enregistrement de la DAU,NaN,NaN,"Alphanumérique, 13 caractères",Numéro sur 13 caractères : les 6 premiers cara...,Numéro d'enregistrement de la DAU


In [8]:
def variable_types(ligne):
    if "Alphanumérique" in ligne["Format"]:
        return "string"
    if "Numérique" in ligne["Format"]:
        return "Int64"
    if "Date" in ligne["Format"]:
        return "string"

dico_variables["Format_python"] = dico_variables.apply(variable_types, axis=1)


In [9]:
dtype_map = dict(zip(dico_variables["Description de la variable"], dico_variables["Format_python"]))
dtype_map = {col : python_type for col, python_type in dtype_map.items() if col in df.columns}
df = df.astype(dtype_map, errors="ignore")


In [10]:
for c in df.columns:
    print(c)

Unnamed: 0
Code de la région du lieu des travaux - Code de la zone
Code de la région du lieu des travaux - Libellé de la zone
Code du département du lieu des travaux - Code de la zone
Code du département du lieu des travaux - Libellé de la zone
Code de la commune du lieu des travaux
Type de DAU -- demande d'autorisation d'urbanisme (PC ou DP généralement, rarement PA)
Numéro d'enregistrement de la DAU
Etat d'avancement du projet
Date réelle d'autorisation (PC) ou de non-opposition (DP) initiale
Date réelle d'ouverture de chantier
Date réelle d'achèvement des travaux
Année de dépôt de la DAU
Date (mois) de prise en compte (DPC) du premier évènement reçu dans Sitadel (dépôt de la demande ou autorisation)
Date (mois) de prise en compte (DPC) de l'autorisation initiale
Date (mois) de prise en compte (DPC) de l'ouverture du chantier
Date (mois) (DPC) de dernière mise à jour des données
Code d'activité principale de l'établissement d'un demandeur avéré en tant que personne morale
Catégorie j

In [11]:
for i in df.dtypes:
    print(i)

col_from_geocodage = [i for i in df.columns if "result_" in i]

int64
object
object
object
object
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
string
object
object
object
object
object
object
object
object
string
string
string
string
string
string
string
string
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
object
string
object
float64
float64
float64
float64
object
object
object
object
object
object
float64
object
object
object
float64
object
object
object


In [12]:
df.dtypes.to_frame()

,0
Unnamed: 0,int64
Code de la région du lieu des travaux - Code de la zone,object
Code de la région du lieu des travaux - Libellé de la zone,object
Code du département du lieu des travaux - Code de la zone,object
Code du département du lieu des travaux - Libellé de la zone,object
...,...
result_citycode,object
result_oldcitycode,float64
result_oldcity,object
result_district,object


In [13]:

nature_batiments = [x for x in df.columns 
                   if ("issue d'une transformation" in x 
                   or "nouvelle construite" in x)
                   and ("locaux non résidentiels" not in x)]

dates_lieux_travaux = ['Numéro de voie du terrain',
       'Type de voie du terrain', 'Libellé de la voie du terrain',
       'Lieu-dit du terrain', 'Localité du terrain', 'Code postal du terrain','Superficie du terrain',"Date (mois) de prise en compte (DPC) du premier évènement reçu dans Sitadel (dépôt de la demande ou autorisation)"]

colonnes_pertinentes =  dates_lieux_travaux + nature_batiments

In [14]:
df.loc[:,colonnes_pertinentes].sample(150)

,Numéro de voie du terrain,Type de voie du terrain,Libellé de la voie du terrain,Lieu-dit du terrain,Localité du terrain,Code postal du terrain,Superficie du terrain,Date (mois) de prise en compte (DPC) du premier évènement reçu dans Sitadel (dépôt de la demande ou autorisation),Surface de plancher de la destination Habitation nouvelle construite,Surface de plancher de la destination Habitation issue d'une transformation,...,Surface de plancher de la destination Artisanat nouvelle construite,Surface de plancher de la destination Artisanat issue d'une transformation,Surface de plancher de la destination Industrie nouvelle construite,Surface de plancher de la destination Industrie issue d'une transformation,Surface de plancher de la destination Exploitation agricole ou forestière nouvelle construite,Surface de plancher de la destination Exploitation agricole ou forestière issue d'une transformation,Surface de plancher de la destination Entrepôt nouvelle construite,Surface de plancher de la destination Entrepôt issue d'une transformation,Surface de plancher de la destination Services publics ou d'intérêt collectif nouvelle construite,Surface de plancher de la destination Services publics ou d'intérêt collectif issue d'une transformation
140006,<NA>,<NA>,<NA>,Quenequen Bras,Loguivy-Plougras,22780.0,1260,2013-04,0.0,0.0,...,0.0,0.0,0,0.0,45.0,0.0,0.0,0.0,0.0,0.0
452577,<NA>,<NA>,BOULEVARD MENDES FRANCE,CC CARREFOUR,LORIENT,56100,2498,2019-03,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
384355,<NA>,<NA>,<NA>,Le Boucle,Beduer,46100.0,70552,2015-05,0.0,0.0,...,0.0,0.0,0.0,0.0,1566.0,0.0,0.0,0.0,0.0,0.0
486797,<NA>,<NA>,RAYAGE DU BOUT,<NA>,Pont-sur-Sambre,59138,7976,2020-11,0.0,0.0,...,0.0,0.0,0.0,0.0,190.0,0.0,0.0,0.0,0.0,0.0
182588,<NA>,<NA>,rue Du General de Gaulle,<NA>,PONT-DE-L'ARCHE,27340.0,0,2018-02,66.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152286,1340,<NA>,Route de Georges,<NA>,BERGERAC,24100.0,63076,2024-12,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,794.0,0.0
723937,<NA>,RUE,Chambourdon,<NA>,Naintre,86530,14051,2013-04,0.0,0.0,...,0.0,0.0,395,0.0,0.0,0.0,0.0,0.0,0.0,0.0
596362,<NA>,<NA>,Avenue de Chalon,<NA>,SAINT-MARCEL,71380,1019,2014-08,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
241923,<NA>,<NA>,<NA>,A ENGALIN,Seysses-Saves,32130.0,9631,2017-01,0.0,0.0,...,0.0,0.0,0.0,0.0,1907.0,0.0,0,0.0,0.0,0.0


In [15]:
df.loc[:,colonnes_pertinentes].sample(100)


def identification_usage(ligne):
    col_max = ligne.iloc[len(dates_lieux_travaux):].idxmax()
    pattern = r"Surface de plancher de la destination (.+?) (nouvelle construite|issue d'une transformation)$"

    match = re.search(pattern, col_max, re.IGNORECASE)
    print("Testing column:", repr(col_max))
    return match.group(1) if match else None

df_usage = df.loc[:,colonnes_pertinentes].sample(1000)
df_usage["Destination du lieu"]= df_usage.apply(identification_usage, axis = 1)



Testing column: 'Surface de plancher de la destination Commerce nouvelle construite'
Testing column: 'Surface de plancher de la destination Entrepôt nouvelle construite'
Testing column: "Surface de plancher de la destination Artisanat issue d'une transformation"
Testing column: 'Surface de plancher de la destination Exploitation agricole ou forestière nouvelle construite'
Testing column: 'Surface de plancher de la destination Entrepôt nouvelle construite'
Testing column: "Surface de plancher de la destination Services publics ou d'intérêt collectif nouvelle construite"
Testing column: "Surface de plancher de la destination Services publics ou d'intérêt collectif issue d'une transformation"
Testing column: 'Surface de plancher de la destination Bureau nouvelle construite'
Testing column: "Surface de plancher de la destination Habitation issue d'une transformation"
Testing column: 'Surface de plancher de la destination Industrie nouvelle construite'
Testing column: 'Surface de plancher d

In [16]:
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786739 entries, 0 to 786738
Columns: 119 entries, Unnamed: 0 to result_status
dtypes: float64(6), int64(1), object(77), string(35)
memory usage: 714.3+ MB


In [17]:
df_usage.head()

,Numéro de voie du terrain,Type de voie du terrain,Libellé de la voie du terrain,Lieu-dit du terrain,Localité du terrain,Code postal du terrain,Superficie du terrain,Date (mois) de prise en compte (DPC) du premier évènement reçu dans Sitadel (dépôt de la demande ou autorisation),Surface de plancher de la destination Habitation nouvelle construite,Surface de plancher de la destination Habitation issue d'une transformation,...,Surface de plancher de la destination Artisanat issue d'une transformation,Surface de plancher de la destination Industrie nouvelle construite,Surface de plancher de la destination Industrie issue d'une transformation,Surface de plancher de la destination Exploitation agricole ou forestière nouvelle construite,Surface de plancher de la destination Exploitation agricole ou forestière issue d'une transformation,Surface de plancher de la destination Entrepôt nouvelle construite,Surface de plancher de la destination Entrepôt issue d'une transformation,Surface de plancher de la destination Services publics ou d'intérêt collectif nouvelle construite,Surface de plancher de la destination Services publics ou d'intérêt collectif issue d'une transformation,Destination du lieu
375138,<NA>,RUE,du Docteur A. Chene,<NA>,Varades,44370.0,433,2013-01,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Commerce
52077,<NA>,<NA>,<NA>,A CASTILLOU,Festes-et-Saint-Andre,11300,18855,2015-04,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,160.0,0.0,0.0,0.0,Entrepôt
560846,43,<NA>,RUE SAINT NICOLAS,<NA>,SAVERNE,67700.0,1579,2015-05,0.0,0.0,...,316.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Artisanat
681827,660,CHEM,de Begue,<NA>,Bressols,82710,103321,2013-09,0.0,0.0,...,0.0,0.0,0.0,869.0,0.0,0.0,0.0,0.0,0.0,Exploitation agricole ou forestière
91427,2,<NA>,rue philippe Lebon,<NA>,DOUVRES-LA-DELIVRANDE,14440.0,0,2021-06,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,136.0,0.0,0.0,0.0,Entrepôt


In [18]:
df["Destination principale"].value_counts()

Destination principale
7                         165977
9                         157836
1                         129660
4                          84580
8                          82167
3                          65829
6                          44744
5                          43730
2                          12215
DESTINATION_PRINCIPALE         1
Name: count, dtype: Int64

In [19]:
df.loc[:,col_from_geocodage]


,result_score,result_score_next,result_label,result_type,result_id,result_housenumber,result_name,result_street,result_postcode,result_city,result_context,result_citycode,result_oldcitycode,result_oldcity,result_district,result_status
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not-found
1,0.852176,0.597063,L'Abergement-Clémenciat,municipality,01001,NaN,L'Abergement-Clémenciat,NaN,1400.0,L'Abergement-Clémenciat,"01, Ain, Auvergne-Rhône-Alpes",01001,NaN,NaN,NaN,ok
2,0.779841,NaN,203 Rue des 5 Chemins 01400 L'Abergement-Cléme...,housenumber,01001_0110_00203,203,203 Rue des 5 Chemins,Rue des 5 Chemins,1400.0,L'Abergement-Clémenciat,"01, Ain, Auvergne-Rhône-Alpes",01001,NaN,NaN,NaN,ok
3,0.903833,0.583818,172 Route du Château 01400 L'Abergement-Clémen...,housenumber,01001_0095_00172,172,172 Route du Château,Route du Château,1400.0,L'Abergement-Clémenciat,"01, Ain, Auvergne-Rhône-Alpes",01001,NaN,NaN,NaN,ok
4,0.601548,NaN,Chemin sous Brody 01400 L'Abergement-Clémenciat,street,01001_0365,NaN,Chemin sous Brody,Chemin sous Brody,1400.0,L'Abergement-Clémenciat,"01, Ain, Auvergne-Rhône-Alpes",01001,NaN,NaN,NaN,ok
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,not-found
786735,0.503825,0.489163,8 route de combani 97680 Tsingoni,housenumber,97617_0260_00008,8.0,8 route de combani,route de combani,97680.0,Tsingoni,"976, Mayotte",97617.0,NaN,NaN,NaN,ok
786736,0.307265,NaN,Quartier Dzaoudzi 97680 Tsingoni,locality,97617_A130,NaN,Quartier Dzaoudzi,NaN,97680.0,Tsingoni,"976, Mayotte",97617.0,NaN,NaN,NaN,ok
786737,0.620043,0.433151,Mroale 97680 Tsingoni,street,97617_rald1b,NaN,Mroale,Mroale,97680.0,Tsingoni,"976, Mayotte",97617.0,NaN,NaN,NaN,ok


In [ ]:
df[df["result_status"] == "error"].loc[:,"Adresse_complete"]


277759                  LE VILLAGE Saint-Genies-de-Varensal
277760                   PLAISANCE Saint-Genies-de-Varensal
277761                   Plaisance Saint-Genies-de-Varensal
277762    RTE DEPARTEMENTALE 22E14 LE VILLAGE Saint-Geni...
277763    1bis RUE des Aires Hautes Saint-Genies-de-Font...
277764               RUE DES HORTS Saint-Genies-de-Fontedit
277765          RUE DE LA VICTOIRE Saint-Genies-de-Fontedit
374514         8 Rue de la Baudiniere THOUARiA_oe-SUR-LOIRE
374520              13 Rue de la BaudiniiA_oere THOUARiA_oe
374521                     rue de Milan THOUARi__-SUR-LOIRE
374522            Avenue de la Baudiniere THOUARE-SUR-LOIRE
374523                  4 Rue du Danube THOUARi_ -SUR-LOIRE
374524                  21 Rue de la Baudinii__re THOUARi__
374525      9001 Avenu du Parc des Sports THOUARE SUR LOIRE
374526                      Le Chene-Vert Thouare-sur-Loire
417855                      ZA  de la VERANGERIE Picauville
417858                    Le Coudray Hou

In [39]:
df["Type principal des locaux d'origine transformés"].value_counts()

Type principal des locaux d'origine transformés
1.0                                49456
4.0                                17054
8.0                                13608
3.0                                12599
9.0                                 9594
7.0                                 8754
5.0                                 5218
6.0                                 3077
2.0                                 1713
1                                    382
8                                    151
7                                    137
4                                    128
5                                     95
9                                     89
3                                     64
6                                     50
2                                     20
TYPE_PRINCIP_LOCAUX_TRANSFORMES        1
Name: count, dtype: Int64